<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/SVD_Recommender/MovieLens_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> SVD Recommender</font></h1>

---

<center>
  <img src="https://bobliu.io/assets/img/cards.509a5045.jpg" width="800" height="300">
</center>


## Objetivo de este Notebook

1. Cargar y preprocesar un Dataset.
2. Realizar un sistema de recomendación basado en Pearson Similarity.
3. Comprobar el performance del sistema.

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar el Dataset</a>  
6. <a href="#item34">Entrenamiento del modelo</a>  
6. <a href="#item34">Validación del modelo</a>  

</font>
</div>

## 1. Contexto


El conjunto de datos MovieLens es uno de los conjuntos de datos de recomendación más populares y ampliamente utilizados en la investigación de sistemas de recomendación. Fue creado por el GroupLens Research Project en la Universidad de Minnesota para impulsar la investigación en sistemas de recomendación, proporcionando un recurso valioso para la comunidad académica y promoviendo el desarrollo y la comprensión de tecnologías de recomendación personalizada.


<b>Descripción de datos</b>

El conjunto de datos MovieLens contiene información sobre:

<b>Películas:</b> Detalles sobre las películas, incluyendo su título, género y año de lanzamiento.

<b>Usuarios:</b> Perfiles de los usuarios que han calificado y/o etiquetado las películas, incluyendo su ID y otros detalles demográficos opcionales.

<b>Calificaciones:</b> Calificaciones numéricas que los usuarios asignan a las películas en una escala de 1 a 5.

<b>Etiquetas:</b> Palabras clave o tags proporcionados por los usuarios para describir el contenido o la esencia de las películas.

El conjunto de datos es ampliamente utilizado con fines académicos y de investigación, siendo una referencia en el diseño y evaluación de sistemas de recomendación de películas. También es útil para el análisis de tendencias y comportamientos en la visualización de películas y la interacción del usuario con el contenido.

<strong>Puede consultar este [link](https://grouplens.org/datasets/movielens/) para leer más sobre la fuente de datos MovieLens proporcionada por GroupLens Research en la Universidad de Minnesota.</strong>

## 2. Descargar y preparar Dataset

In [ ]:
# Descargar el dataset Movielens
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
!unzip dataset.zip
!ls -la

In [3]:
# Principales librerías
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # Turn off warnings


In [4]:
links   = pd.read_csv("ml-latest-small/links.csv")
movies  = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")
tags    = pd.read_csv("ml-latest-small/tags.csv")


In [ ]:
links.head()

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
tags.head()

In [ ]:
print("  Movies: {} \n  Ratings: {}".format(len(movies), len(ratings)))


In [10]:
# Fusiona ambos datasets basados en la columna 'movieId'
data = pd.merge(ratings, movies, on='movieId')

In [11]:
movie_titles = data['title'].unique().tolist()
movie_ids = data['movieId'].unique().tolist()


In [12]:
# Crear matriz pivotada de usuarios y películas
user_movie_rating = data.pivot_table(index='userId', columns='title', values='rating')


In [13]:
#500 películas más vistas
movies_pop = user_movie_rating.isnull().sum().sort_values()[:500]


In [14]:
user_movie_rating = user_movie_rating[movies_pop.index.tolist()]

In [15]:
user_movie_rating = user_movie_rating.reset_index()

Muestreo (Enmascaramiento)

In [ ]:
user_movie_rating

In [100]:
from sklearn.model_selection import train_test_split

# Convertir la matriz pivotada en un DataFrame y dividir en train y test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [104]:
train_data_matrix = train_data.pivot_table(index='userId', columns='title', values='rating')
test_data_matrix = test_data.pivot_table(index='userId', columns='title', values='rating')

train_data_matrix = train_data_matrix[movies_pop.index.tolist()].reset_index()
test_data_matrix = test_data_matrix[movies_pop.index.tolist()].reset_index()

In [105]:
# Volver a crear matrices pivotadas para entrenamiento y prueba
train_data_matrix = train_data_matrix.fillna(0)
test_data_matrix = test_data_matrix.fillna(0)

In [ ]:
test_data_matrix.head()

## 3. SVD (Singular Value Decomposition)

Aplicaremos el enfoque model based basado en SVD

In [107]:
from numpy.linalg import svd

# Descomponemos la matriz de entrenamiento usando SVD
U, sigma_values, Vt = svd(train_data_matrix.drop(columns = ['userId']), full_matrices=False)

# La matriz sigma devuelta es solo una lista de valores singulares. La convertimos a una matriz diagonal.
sigma = np.diag(sigma_values)


In [108]:
# Predicciones con el modelo
predicted_ratings = np.dot(np.dot(U, sigma), Vt)


In [ ]:
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=train_data_matrix.drop(columns = ['userId']).columns, index=train_data_matrix.index)
predicted_ratings_df['userId'] = train_data_matrix['userId']
predicted_ratings_df.head()


Predicciones

In [110]:
# Seleccionar un usuario (por ejemplo, el usuario con ID 82)
user_idx = 72
user_predictions = predicted_ratings_df[predicted_ratings_df.userId == user_idx]

In [111]:
# Peliculas calificadas por el cliente

rated_movies_by_user = train_data_matrix[train_data_matrix.userId == user_idx]
already_rated = rated_movies_by_user[rated_movies_by_user > 0].index.tolist()

In [ ]:
pddf_rated_movies_by_user = rated_movies_by_user.T.reset_index()
pddf_rated_movies_by_user.columns = ['title', 'rating']
pddf_rated_movies_by_user = pddf_rated_movies_by_user[pddf_rated_movies_by_user.rating.between(1, 5)]
pddf_rated_movies_by_user.sort_values(by = 'rating', ascending = False, inplace = True)
already_rated = pddf_rated_movies_by_user.title.tolist()

pddf_rated_movies_by_user.head(10)

In [ ]:
# Películas que no ha calificado
movie_recommendations = user_predictions.T.reset_index()
movie_recommendations.columns = ['title', 'rating']
top_recommendations = movie_recommendations[movie_recommendations.title.isin(unrated_movies_by_user)].sort_values(by = 'rating', ascending=False)
top_recommendations.head()

## 4. Evaluación del modelo

MSE

In [ ]:
from sklearn.metrics import *

# Filtramos las predicciones reales
real_ratings = test_data_matrix.drop(columns = ['userId']).values[test_data_matrix.drop(columns = ['userId']).values.nonzero()]
predicted_ratings = predicted_ratings_df.values[test_data_matrix.drop(columns = ['userId']).values.nonzero()]

mse = mean_squared_error(real_ratings, predicted_ratings)
print("MSE en conjunto de entrenamiento:", mse)


In [ ]:
from sklearn.metrics import *

# Filtramos las predicciones reales
real_ratings = train_data_matrix.drop(columns = ['userId']).values[train_data_matrix.drop(columns = ['userId']).values.nonzero()]
predicted_ratings = predicted_ratings_df.values[train_data_matrix.drop(columns = ['userId']).values.nonzero()]

mse = mean_squared_error(real_ratings, predicted_ratings)
print("MSE en conjunto de entrenamiento:", mse)


Evaluación del hit Rate

In [177]:
# Obtiene las películas vistas por cada usuario en entrenamiento

user_seen_movies = {}

for col in range(0, len(train_data_matrix)):
  user = train_data_matrix[train_data_matrix.index == col]
  temp = user.T.reset_index()
  temp.columns = ['tittle', 'rating']
  user_seen_movies[col] = temp[temp.rating.between(1,5)].tittle.tolist()

In [178]:
# Obtiene las películas con las calificaciones predichas más altas para cada usuario

predicted_movies = {}

for col in predicted_ratings_df.userId.tolist():
  user_pred = predicted_ratings_df[predicted_ratings_df.userId == col]
  temp = user_pred.T.reset_index()
  temp.columns = ['tittle', 'rating']
  recs = temp[~temp.tittle.isin(user_seen_movies.get(col, []) + ['userId'])]
  top_recs = recs.sort_values(by = 'rating', ascending = False).head(10)
  predicted_movies[col] = top_recs.tittle.tolist()


In [179]:
# Obtiene las películas vistas por cada usuario en test y que haya disfrutado

user_seen_movies_test = {}

for col in range(0, len(test_data_matrix)):
  user = test_data_matrix[test_data_matrix.index == col]
  temp = user.T.reset_index()
  temp.columns = ['tittle', 'rating']
  user_seen_movies_test[col] = temp[temp.rating.between(4,5)].tittle.tolist()

In [ ]:
intersectan = 0

for col in user_seen_movies_test.keys():
  vistas = set(user_seen_movies_test[col])
  recomendadas = set(predicted_movies.get(col, []))
  # Verificar si hay intersección
  intersectan += not recomendadas.isdisjoint(vistas)

print('El hit rate de recomendaciones en usuarios en test es de :', round(intersectan/len(user_seen_movies_test.keys())*100), '%')

---
## Gracias por completar este laboratorio!